In [2]:
!pip list

Package           Version
----------------- -----------
asttokens         3.0.0
colorama          0.4.6
comm              0.2.2
debugpy           1.8.12
decorator         5.2.1
executing         2.2.0
filetype          1.2.0
flatbuffers       25.2.10
ipykernel         6.29.5
ipython           8.32.0
jedi              0.19.2
jupyter_client    8.6.3
jupyter_core      5.7.2
matplotlib-inline 0.1.7
mmh3              4.1.0
mpmath            1.3.0
nest-asyncio      1.6.0
packaging         24.2
parso             0.8.4
pillow            10.4.0
pip               25.0.1
platformdirs      4.3.6
portalocker       2.10.1
prompt_toolkit    3.0.50
propcache         0.3.0
protobuf          5.29.3
psutil            7.0.0
pure_eval         0.2.3
py_rust_stemmers  0.1.5
pycparser         2.22
pydantic_core     2.27.2
Pygments          2.19.1
pypdf             5.3.0
pyreadline3       3.5.4
python-dateutil   2.9.0.post0
python-dotenv     1.0.1
python-iso639     2025.2.18
python-magic      0.4.27
pywin32   

In [3]:
!pip install -r requirements.txt

  Using cached langchain_qdrant-0.2.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_community-0.3.18-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain-0.3.19-py3-none-any.whl.metadata (7.9 kB)
  Using cached qdrant_client-1.13.2-py3-none-any.whl.metadata (10 kB)
  Using cached langchain_ollama-0.2.3-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.10-py3-none-any.whl.metadata (14 kB)
  Using cached fastembed-0.5.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached unstructured-0.16.23-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_core-0.3.37-py3-none-any.whl.metadata (5.9 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached SQLAlchemy-2.0.38-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached aiohttp-3.11.12-c

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\shaoe\\OneDrive\\Desktop\\haliqcode\\.venv\\Lib\\site-packages\\cryptography\\hazmat\\primitives\\kdf\\argon2.py'
Check the permissions.



In [4]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from ollama import chat
import ollama
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models
from langchain_ollama import OllamaEmbeddings
from langsmith import wrappers, traceable

c:\Users\shaoe\OneDrive\Desktop\haliqcode\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.environ["LANGSMITH_API_KEY"])

KeyError: 'LANGSMITH_API_KEY'

In [6]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredMarkdownLoader

def load_document(file_path: str):
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    elif file_path.endswith(".md"):
        loader = UnstructuredMarkdownLoader(file_path)
    documents = loader.load()
    return documents

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

@traceable
def create_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        # Separators in order of priority (Semantic Separation)
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
        chunk_size=512,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=False,
        keep_separator=True,
        strip_whitespace=True
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

In [8]:
from langchain_ollama import OllamaEmbeddings
from fastembed import LateInteractionTextEmbedding, SparseTextEmbedding 

def generate_dense_vector(text: str):
    dense_model_path = 'rjmalagon/gte-qwen2-1.5b-instruct-embed-f16'
    dense_model = OllamaEmbeddings(model=dense_model_path)
    dense_vector = dense_model.embed_query(text)
    # print(len(dense_vector)) # Show Dims
    
    return dense_vector

In [9]:
# Move class definition outside the function
class DocumentStruct:
    def __init__(self, metadata, page_content, embedding, sparse_embedding, late_interaction):
        self.metadata = metadata
        self.page_content = page_content
        self.embedding = embedding
        self.sparse_embedding = sparse_embedding
        self.late_interaction = late_interaction
        
    def __repr__(self):
        return f"DocumentEmbedding(metadata={self.metadata}, page_content={self.page_content}, embedding={self.embedding}, sparse_embedding={self.sparse_embedding}, late_interaction={self.late_interaction} )"
    

@traceable
def generateEmbeddings(documents):
    embeddingsList = []
    late_interaction_embedding_model = LateInteractionTextEmbedding("colbert-ir/colbertv2.0")
    sparse_embeddings_model = SparseTextEmbedding("Qdrant/bm25")
    dense_model = OllamaEmbeddings(model='rjmalagon/gte-qwen2-1.5b-instruct-embed-f16')

    for idx, item in enumerate(documents):
        metadata = item.metadata
        page_content = item.page_content
        embedding = dense_model.embed_query(item.page_content)
        sparse_embedding = list(sparse_embeddings_model.embed(item.page_content))
        late_interaction_embed = list(late_interaction_embedding_model.embed(item.page_content))
        embeddingsList.append(DocumentStruct(metadata=metadata, 
                                             page_content=page_content, 
                                             embedding=embedding, 
                                             sparse_embedding=sparse_embedding,
                                             late_interaction=late_interaction_embed
                                             )
                              )
        
        # Each item will have a sparse and dense vector embedding. Which will be then used for hybrid search functions.   
    return embeddingsList

### Figure out how to make this into a Langsmith Traceable Chain

In [14]:
document_path = "C:/Users/shaoe/OneDrive/Desktop/haliqcode/The_Hundred_page_Machine_Learning_Book_Andriy_Burkov_Z_Library.pdf"

docs = load_document(document_path)
chunked_docs = create_chunks(docs)
result = generateEmbeddings(chunked_docs)

### Upload to DB Pipeline

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(
    url=os.environ["QDRANT_URL"], 
    api_key=os.environ["QDRANT_API_KEY"],
)


In [16]:
client.delete_collection("Ollama-RAG")

True

In [17]:
from qdrant_client.models import Distance, VectorParams, models
from qdrant_client.models import Distance, VectorParams, models

client.create_collection(
    "Ollama-RAG",
    vectors_config={
        "gte-qwen1.5": models.VectorParams(
            size=1536,
            distance=models.Distance.COSINE,
        ),
        "colbertv2.0": models.VectorParams(
            size=128,
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM,
            )
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(modifier=models.Modifier.IDF
        )
    }
)

True

In [18]:
from qdrant_client.models import PointStruct

def prepare_vector_points(embeddingsList):
    points = []
    
    for idx, item in enumerate(embeddingsList):
        point = PointStruct(
            id = idx,
            vector = {
                "gte-qwen1.5": item.embedding,
                "bm25": models.SparseVector(indices=item.sparse_embedding[0].indices, values=item.sparse_embedding[0].values),
                "colbertv2.0": item.late_interaction[0],
            },
            payload={ # Payload: Provide context for points
                    "text": item.page_content,
                    "metadata": {
                        "source": item.metadata['source'],
                        "page": item.metadata['page'],
                    }
                },
        )
        points.append(point)
    return points

In [19]:
total = prepare_vector_points(result)

In [20]:
total[0]

PointStruct(id=0, vector={'gte-qwen1.5': [0.006369679, 0.07534177, -0.018120188, 0.01558661, 0.0019456361, 0.009437969, -0.0068539954, -0.021222483, -0.0003420979, 0.0075301626, 0.028386017, -0.037214607, -0.019489557, 0.020202538, 0.0104253115, 0.014907465, 0.014992168, -0.0068104924, 0.020848509, 0.058754258, 0.016272431, -0.0011689797, -0.048300017, -0.0021263582, -0.019344902, -0.017234983, 0.008372368, 0.019177813, 0.060724307, -0.026384518, -0.021740044, 0.012975712, -0.018820886, 0.09527176, -0.029280072, -0.008937668, 0.0020321326, -0.027254816, 0.03360931, -0.028222844, 0.004176389, 0.03139519, 0.006667968, 0.020230342, -0.0027458868, -0.00093986833, -0.060961526, -0.012917651, -0.017216343, 0.039809838, -0.0015730072, -0.027420009, -0.004121382, -0.012329858, 0.04858852, -0.015060896, 0.024012977, -0.005048262, 0.0008881803, -0.0016924114, 0.023372244, 0.028588038, -0.04721799, 0.0027444272, 0.013888919, 0.024408322, -0.027988011, -0.016865863, 0.037340157, 0.013730644, 0.013

In [ ]:
# size of vector too big to fit 
client.upsert(
    collection_name="Ollama-RAG",
    points=total
)

In [23]:
# Try this approach
batch_size = 10  # Start with a smaller number and adjust as needed
total_points = total  # Your original points list

for i in range(0, len(total_points), batch_size):
    batch = total_points[i:i+batch_size]
    try:
        print(f"Upserting batch {i//batch_size + 1}/{(len(total_points) + batch_size - 1)//batch_size}")
        client.upsert(
            collection_name="Ollama-RAG",
            points=batch,
            wait=True  # Ensure each batch completes before proceeding
        )
    except Exception as e:
        print(f"Error with batch {i//batch_size + 1}: {str(e)}")
        # Optional: add retry logic here

Upserting batch 1/67
Upserting batch 2/67
Upserting batch 3/67
Upserting batch 4/67
Upserting batch 5/67
Upserting batch 6/67
Upserting batch 7/67
Upserting batch 8/67
Upserting batch 9/67
Upserting batch 10/67
Upserting batch 11/67
Upserting batch 12/67
Upserting batch 13/67
Upserting batch 14/67
Upserting batch 15/67
Upserting batch 16/67
Upserting batch 17/67
Upserting batch 18/67
Upserting batch 19/67
Upserting batch 20/67
Upserting batch 21/67
Upserting batch 22/67
Upserting batch 23/67
Upserting batch 24/67
Upserting batch 25/67
Upserting batch 26/67
Upserting batch 27/67
Upserting batch 28/67
Upserting batch 29/67
Upserting batch 30/67
Upserting batch 31/67
Upserting batch 32/67
Upserting batch 33/67
Upserting batch 34/67
Upserting batch 35/67
Upserting batch 36/67
Upserting batch 37/67
Upserting batch 38/67
Upserting batch 39/67
Upserting batch 40/67
Upserting batch 41/67
Upserting batch 42/67
Upserting batch 43/67
Upserting batch 44/67
Upserting batch 45/67
Upserting batch 46/

In [24]:
# Loading doc function, recursive text spliter